In [1]:
import sys, os, json, torch

# Add project root to path so model imports work
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from argparse import Namespace
from models.inp import INP_MedClassification
from models.loss import ELBOLoss

from open_clip import create_model_and_transforms, get_tokenizer
from open_clip.factory import HF_HUB_PREFIX, _MODEL_CONFIGS

print("✓ All imports successful")

/usr/bmicnas03/data-biwi-01/ldrole_student/data/miniforge3/envs/inps/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/bmicnas03/data-biwi-01/ldrole_student/data/miniforge3/envs/inps/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/bmicnas03/data-biwi-01/ldrole_student/data/miniforge3/envs/inps/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


✓ All imports successful


/usr/bmicnas03/data-biwi-01/ldrole_student/data/miniforge3/envs/inps/lib/python3.11/site-packages/timm/models/layers/__init__.py:49: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
# ── Load BiomedCLIP ──
LOCAL_DIR = "/home/ldrole/my_space/work/cam_phd/checkpoints/biomedclip"
MODEL_NAME = "biomedclip_local"

with open(f"{LOCAL_DIR}/open_clip_config.json", "r") as f:
    clip_config = json.load(f)
    model_cfg = clip_config["model_cfg"]
    preprocess_cfg = clip_config["preprocess_cfg"]

if (not MODEL_NAME.startswith(HF_HUB_PREFIX)
    and MODEL_NAME not in _MODEL_CONFIGS
    and clip_config is not None):
    _MODEL_CONFIGS[MODEL_NAME] = model_cfg

tokenizer = get_tokenizer(MODEL_NAME)
clip_model, _, preprocess = create_model_and_transforms(
    model_name=MODEL_NAME,
    pretrained=f"{LOCAL_DIR}/open_clip_pytorch_model.bin",
    **{f"image_{k}": v for k, v in preprocess_cfg.items()},
)
clip_model.eval()

# ── Config ──
med_config = Namespace(
    n_ways=3,
    hidden_dim=128,
    clip_dim=512,
    train_num_z_samples=1,
    test_num_z_samples=4,
)

# ── Instantiate model & loss ──
model = INP_MedClassification(med_config, clip_model, tokenizer)
loss_fn = ELBOLoss(categorical=True)

print(f"✓ INP_MedClassification created  (d={med_config.hidden_dim}, n_ways={med_config.n_ways})")
print(f"✓ ELBOLoss created  (categorical=True)")

/usr/bmicnas03/data-biwi-01/ldrole_student/data/miniforge3/envs/inps/lib/python3.11/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/bmicnas03/data-biwi-01/ldrole_student/data/miniforge3/envs/inps/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


✓ INP_MedClassification created  (d=128, n_ways=3)
✓ ELBOLoss created  (categorical=True)
